# V2: SCF optimization with VAMPyR

## V2.1: Hydrogen atom

In order to solve the one-electron Schr\"{o}dinger equation in MWs we reformulate them in an integral form [1].

\begin{equation}
 \phi = -2\hat{G}_{\mu}\hat{V}\phi
\end{equation}

Where $\hat{V}$ is the potential acting on the system, $\phi$ is the wavefunction, $\hat{G}$ is the Helmholtz integral operator, where its kernel is defined as $G_\mu(r - r') = \frac{\exp(-\mu |r - r'|)}{4\pi |r - r'|}$
and $\mu$ is a parameter defined above through the energy.

The Helmholtz operator is already implemented in vampyr, therefore the only things you need are the integral KS equation and the definition of $\mu$ 

\begin{equation}
     \mu = \sqrt{-2E}
\end{equation}

The way you initialize the Helmholtz operator is as follows
```
H = vp.HelmholtzOperator( mra, exp=mu, prec=eps )
```
where `mu` is the $\mu$ is the parameter defined above, mra you have seen before, and `eps` is the desired threshold precision. This operator is applied the same way you applied the vp.ScalingProjector earlier.

In this exercise you will be solving the KS equation iteratively for a simple system, the Hydrogen atom. This means that you only have the nuclear potential to take into account for the potential term in the KS equation.

$$    V_{nuc}(\mathbf{r}) = -\frac{1}{|\mathbf{r}|}$$

We will also be working with a single orbital, of which the initial guess is

$$    \phi_0(\mathbf{r}) = e^{-|\mathbf{r}|^2} $$
where 
$$    |\mathbf{r}| = \sqrt{x^2 + y^2 + z^2}$$


The orbital update is defined as follows
\begin{align}
    \Delta\tilde{\phi}^n &= -2\hat{G}[V_{nuc}\phi^n] - \phi^n \\
    \Delta\tilde{\phi}^n &= \tilde{\phi}^{n+1} - \phi^n
\end{align}
where we use \~ to denote a function that is **not** normalized, and $n$ is the iteration index.

#### Implementation exercise:

1. Make a nuclear potential as a python function `f_nuc(r)`
2. Make an initial guess for the orbital as a python function `f_phi(r)` (hint use `np.exp` to get an exponetial function)
3. Create a Helmholtz operator $G_\mu$ with $\mu$ as shown above, use the exact value of $E = -0.5 a.u.$ for a hydrogen atom
4. Project both nuclear potential ($V$) and orbital ($\phi_n$) to the MW basis using a `vp.ScalingProjector` with precision $\epsilon=1.0e-3$
5. Compute new orbital through application of the Helmholtz operator
6. Compute the size of the orbital update $||\tilde{\phi}^{n+1} - \phi^n||$
7. Normalize the orbital $\phi^{n+1} = \tilde{\phi}^{n+1}/||\tilde{\phi}^{n+1}||$
8. Update orbital $\phi^{n+1} \rightarrow \phi^{n}$ for next iteration
9. Repeat steps 5-8 until your wavefunction has converged

The convergence criterion is the norm of $\Delta \phi^n$, but you should start by looping a set amount of times before trying the threshold.

In [ ]:
from vampyr import vampyr3d as vp
import numpy as np
import matplotlib.pyplot as plt

r_x = np.linspace(-0.99, 0.99, 1000) # create an evenly spaced set of points between -0.99 and 0.99
r_y = np.zeros(1000)
r_z = np.zeros(1000)
r = [r_x, r_y, r_z]

# Analytic nuclear potential
def f_nuc(r):
    # TODO: implement the nuclear potential
    return

# Analytic guess for solution
def f_phi(r):
    # TODO: implement the initial guess for the orbital
    return

# Prepare Helmholtz operator
E = -0.5
mu = np.sqrt(-2*E)
G_mu =   # TODO: Construct BSH operator from mu)

V =      # TODO: Project nuclear potential V from f_nuc
phi_n =  # TODO: Project starting guess phi_n from f_phi
phi_n.normalize()

# Optimization loop
thrs = 1.0e-3
update = 1.0
i = 0
while (i > 10): # switch to (update > thrs) later
    # TODO:
    # Compute product of potential V and wavefunction phi_n
    # Apply Helmholtz operator to obtain phi_np1
    # Compute norm = ||phi^{n+1}||
    # Compute update = ||phi^{n+1} - phi^{n}||
   
    # this will plot the wavefunction at each iteration
    phi_n_plt = [phi_n([x, 0.0, 0.0]) for x in r_x]
    plt.plot(r_x, phi_n_plt) 
    
    # this will print some info, you need to compute in the loop:
    print("iteration: {}    Norm: {}   Update: {}".format(i, norm, update))
    i += 1

plt.show()

## V2.2 Extension to Helium

A few things change when you go from Hydrogen to Helium:
1. The energy is no longer known exactly, and thus will have to be computed from the wavefunction
2. The Helmholtz operator which depends on the energy through $\mu = \sqrt{-2E}$ needs to be updated in every iteration
3. The potential operator $V$ depends on the wavefunction and must be updated in every iteration

In this example we will use the Hartree-Fock model, which for a single-orbital system like Helium, reduces to the following potential operator:
\begin{align}
  \hat{V} &= \hat{V}_{nuc} + 2\hat{J} - \hat{K}\\
          &= \hat{V}_{nuc} + \hat{J}
\end{align}
since $\hat{K} = \hat{J}$ for a doubly occupied single orbital.

The Coulomb potential $\hat{J}$ can be computed by application of the Poisson operator $P$:
\begin{equation}
  \hat{J}(r) = P\left[4\pi\rho\right]
\end{equation}
Where $\rho$ is the square of the orbital
\begin{equation}
  \rho = \phi*\phi
\end{equation}

#### Pen and paper exercise:

Use the fact that
\begin{equation}
  \tilde{\phi}^{n+1} = -\Big[\hat{T} - E^n\Big]^{-1} V^n\phi^n \end{equation}
to show that
\begin{equation}
  E^{n+1} = \frac{\langle\tilde{\phi}^{n+1}|\hat{T} +
            \hat{V}^{n+1}|\tilde{\phi}^{n+1}\rangle}
            {||\tilde{\phi}^{n+1}||^2}
\end{equation}
can be written as a pure update $dE^n$ involving only the potentials $\hat{V}^{n+1}$, $\hat{V}^n$ as well as the orbitals $\tilde{\phi}^{n+1}$ and $\phi^n$
\begin{equation}
  E^{n+1} = E^{n} + dE^n
\end{equation}


#### Implementation exercise:
1. Make a nuclear potential function `f_nuc(r)` for the Helium atom
2. Make an initial guess for the orbital as a python function `f_phi(r)` (hint use `np.exp` to get an exponetial function)
3. Project both nuclear potential ($V$) and orbital ($\phi_n$) to the MW basis using a `vp.ScalingProjector` with precision $\epsilon=1.0e-3$
4. Create a Helmholtz operator $G^n$ with $\mu^n$ using the current energy $E^n$
5. Compute total potential $\hat{V^n} = \hat{V}_{nuc} + \hat{J^n}$, where the Coulomb potential is computed using the `vp.PoissonOperator` on the current squared orbital $\rho^n = ||\phi^n||^2$
6. Compute new orbital through application of the Helmholtz operator on $\phi^{n+1} = -2\hat{G}^n\hat{V}^n\phi^n$
7. Compute the size of the orbital update $||\tilde{\phi}^{n+1} - \phi^n||$
8. Normalize the orbital $\phi^{n+1} = \tilde{\phi}^{n+1}/||\tilde{\phi}^{n+1}||$
9. Update orbital $\phi^{n+1} \rightarrow \phi^{n}$ for next iteration
10. Repeat steps 4-9 until your wavefunction has converged

The convergence criterion is the norm of $\Delta \phi^n$, but you should start by looping a set amount of times before trying the threshold.

In [ ]:
from vampyr import vampyr3d as vp
import numpy as np
import matplotlib.pyplot as plt

r_x = np.linspace(-0.99, 0.99, 1000) # create an evenly spaced set of points between -0.99 and 0.99
r_y = np.zeros(1000)
r_z = np.zeros(1000)
r = [r_x, r_y, r_z]

# Analytic nuclear potential Helium
def f_nuc(r):
    #implement the nuclear potential
    return

# Analytic guess for solution (same as for Hydrogen)
def f_phi(r):
    # implement the initial guess for the orbital
    return


# TODO:
# Project nuclear potential V_nuc from f_nuc
# Project starting guess phi_n from f_phi
# Set a starting guess E_n for the energy


# Optimization loop
thrs = 1.0e-3
update = 1.0
i = 0
while (i > 10): # switch to (update > thrs) later
   
    # Prepare Helmholtz operator from current energy
    mu_n = np.sqrt(-2*E_n)
    G_n = # TODO: Construct BSH operator from mu_n)
    
    # TODO:
    # Compute rho
    # Initialize vp.PoissonOperator and compute J
    # Compute total potential V = V_nuc + J
    # Iterate Helmholtz operator to get new orbital phi^{n+1}
    
    dE_n = # TODO: insert energy expression from above

    # Prepare for next iteration
    E_n += dE_n
    phi_n += dPhi_n
    
    # This will plot the wavefunction at each iteration
    phi_n_plt = [phi_n([x, 0.0, 0.0]) for x in r_x]
    plt.plot(r_x, phi_n_plt) 
    
    # this will print some info, you need to compute in the loop:
    # norm = ||phi^{n+1}||
    # update = ||phi^{n+1} - phi^{n}||
    print("iteration: {}  Energy: {}    Norm: {}   Update: {}".format(i, E_n, norm, update))
    i += 1
    
plt.show()


##  Sources


[1] Stig Rune Jensen, Santanu Saha, José A. Flores-Livas, William Huhn, Volker Blum, Stefan Goedecker, and Luca Frediani The Elephant in the Room of Density Functional Theory Calculations. The Journal of Physical Chemistry Letters 2017 8 (7), 1449-1457
DOI: 10.1021/acs.jpclett.7b00255
